# NOTEBOOK-PIPELINE: CREATE AND REFRESH MATERIALIZED LAKE VIEWS FOR HR

In [1]:
from typing import Union, List, Dict
from dataclasses import dataclass, field, asdict


@dataclass
class NotebookActivity:
    name: str = None
    path: str = None
    timeoutPerCellInSeconds: int = 600
    args: Dict = field(default_factory=lambda: {})
    retry: int = 0
    retryIntervallInSeconds: int = 10
    dependencies: List[str] = field(default_factory=lambda: [])

    def _prep_properties(self):
        # Dependencies
        for i, dep in enumerate(self.dependencies):
            if isinstance(dep, NotebookActivity):
                self.dependencies[0] = dep.name

        # path
        if self.path is None:
            self.path = self.name
    
    def to_activity(self) -> dict:
        self._prep_properties()
        return asdict(self)


StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 3, Finished, Available, Finished)

## BusinessSilverLayer - atoss

In [2]:
default_args = {
    "IS_DEV": False,
    "FULL_REFRESH": False
}

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 4, Finished, Available, Finished)

In [3]:
atoss_employees_mlv = NotebookActivity(
    name="silver.atoss.employees_mlv",
    args=default_args
)

atoss_employees_demo_mlv = NotebookActivity(
    name="silver.atoss.employees_demo_mlv",
    args=default_args
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 5, Finished, Available, Finished)

In [4]:
atoss_workingtimes_mlv = NotebookActivity(
    name="silver.atoss.workingtimes_mlv",
    args=default_args
)

atoss_workingtimes_demo_mlv = NotebookActivity(
    name="silver.atoss.workingtimes_demo_mlv",
    args=default_args
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 6, Finished, Available, Finished)

In [5]:
atoss_payments_mlv = NotebookActivity(
    name="silver.atoss.payments_mlv",
    args=default_args
)

atoss_payments_demo_mlv = NotebookActivity(
    name="silver.atoss.payments_demo_mlv",
    args=default_args
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 7, Finished, Available, Finished)

## GoldLayer - gfhrdm

In [6]:
gfhrdm_dim_costcenters = NotebookActivity(
    name="gold.gfhrdm.dim_costcenters",
    args=default_args,
    dependencies=[
        atoss_employees_mlv
    ]
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 8, Finished, Available, Finished)

In [7]:
gfhrdm_dim_employees = NotebookActivity(
    name="gold.gfhrdm.dim_employees",
    args=default_args,
    dependencies=[
        atoss_employees_mlv
    ]
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 9, Finished, Available, Finished)

In [8]:
gfhrdm_dim_payment_types = NotebookActivity(
    name="gold.gfhrdm.dim_payment_types",
    args=default_args
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 10, Finished, Available, Finished)

In [9]:
gfhrdm_fact_payments = NotebookActivity(
    name="gold.gfhrdm.fact_payments",
    args=default_args,
    dependencies=[
        atoss_payments_mlv
    ]
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 11, Finished, Available, Finished)

In [10]:
gfhrdm_fact_workingtimes = NotebookActivity(
    name="gold.gfhrdm.fact_workingtimes",
    args=default_args,
    dependencies=[
        atoss_workingtimes_mlv
    ]
)

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 12, Finished, Available, Finished)

## DAG Execution

In [11]:
DAG = {
    "activities": [
        # BusinessSilverLayer
        atoss_employees_mlv.to_activity(),
        atoss_workingtimes_mlv.to_activity(),
        atoss_payments_mlv.to_activity(),

        # BusinessSilverLayer - Demo
        atoss_employees_demo_mlv.to_activity(),
        atoss_workingtimes_demo_mlv.to_activity(),
        atoss_payments_demo_mlv.to_activity(),

        # GoldLayer - gfhrdm
        gfhrdm_dim_costcenters.to_activity(),
        gfhrdm_dim_employees.to_activity(),
        gfhrdm_dim_payment_types.to_activity(),
        gfhrdm_fact_payments.to_activity(),
        gfhrdm_fact_workingtimes.to_activity(),
    ]
}
DAG

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 13, Finished, Available, Finished)

{'activities': [{'name': 'silver.atoss.employees_mlv',
   'path': 'silver.atoss.employees_mlv',
   'timeoutPerCellInSeconds': 600,
   'args': {'IS_DEV': False, 'FULL_REFRESH': False},
   'retry': 0,
   'retryIntervallInSeconds': 10,
   'dependencies': []},
  {'name': 'silver.atoss.workingtimes_mlv',
   'path': 'silver.atoss.workingtimes_mlv',
   'timeoutPerCellInSeconds': 600,
   'args': {'IS_DEV': False, 'FULL_REFRESH': False},
   'retry': 0,
   'retryIntervallInSeconds': 10,
   'dependencies': []},
  {'name': 'silver.atoss.payments_mlv',
   'path': 'silver.atoss.payments_mlv',
   'timeoutPerCellInSeconds': 600,
   'args': {'IS_DEV': False, 'FULL_REFRESH': False},
   'retry': 0,
   'retryIntervallInSeconds': 10,
   'dependencies': []},
  {'name': 'silver.atoss.employees_demo_mlv',
   'path': 'silver.atoss.employees_demo_mlv',
   'timeoutPerCellInSeconds': 600,
   'args': {'IS_DEV': False, 'FULL_REFRESH': False},
   'retry': 0,
   'retryIntervallInSeconds': 10,
   'dependencies': []},


In [12]:
notebookutils.notebook.runMultiple(DAG, {"displayDAGViaGraphviz": True})

StatementMeta(, 83f7782c-a01c-4164-b9b4-1f6e547fee47, 14, Submitted, Running, Running)